In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 

from hts.hierarchy import HierarchyTree
from hts import HTSRegressor

In [2]:
sales = pd.read_csv('/Users/user/hts-forecast/volume/data/raw/sales_train_evaluation.csv')
cal = pd.read_csv('/Users/user/hts-forecast/volume/data/raw/calendar.csv')

In [3]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [76]:
#take the dataframe and only present the relevant category
def filter_aggregate_sales(frame, column_to_filter, column_value_to_filter, column_to_aggregate):
    frame = frame[frame[column_to_filter] == column_value_to_filter]
    frame = frame.groupby(column_to_aggregate).sum().T
    return frame

In [79]:
#merge sales with calendar 
def merge_pd(frame1, frame2, merge_index):
    mergedframe = pd.merge(frame1, frame2, on = merge_index, how = 'inner')
    mergedframe.date = pd.to_datetime(mergedframe.date)
    mergedframe = mergedframe.drop(['wm_yr_wk', 'weekday', 'year', 'd', 'event_name_1', 'event_name_2', 
                                      'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI'], axis='columns')
    return mergedframe

In [5]:
states = filter_aggregate_sales(frame=sales, column_to_filter='state_id', column_value_to_filter='CA',
                                 column_to_aggregate='state_id')

In [80]:
states['total'] = states.CA + states.TX + states.WI

for state in sales.state_id.unique():
    states[state] = filter_aggregate_sales(frame=sales, column_to_filter='state_id', column_value_to_filter=state,
                                                    column_to_aggregate='state_id')

for store in sales.store_id.unique():
    states[store] = filter_aggregate_sales(frame=sales, column_to_filter='store_id', column_value_to_filter=store,
                                                    column_to_aggregate='store_id')    

for category in sales.cat_id.unique():
    states[category] = filter_aggregate_sales(frame=sales, column_to_filter='cat_id', column_value_to_filter=category,
                                                       column_to_aggregate='cat_id') 
for dept in sales.dept_id.unique():
    states[dept] = filter_aggregate_sales(frame=sales, column_to_filter='dept_id', column_value_to_filter=dept,
                                                       column_to_aggregate='dept_id') 

states.head()

state_id,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,...,FOODS,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,FOODS_1,FOODS_2,FOODS_3,total,date
d_1,14195,9438,8998,4337,3494,4739,1625,2556,3852,3030,...,23178,3610,154,4105,1584,2343,4094,16741,32631,NaT
d_2,13805,9630,8314,4155,3046,4827,1777,2687,3937,3006,...,22758,3172,185,3858,1776,2216,4209,16333,31749,NaT
d_3,10108,6778,6897,2816,2121,3785,1386,1822,2731,2225,...,17174,2497,185,2827,1100,1657,3174,12343,23783,NaT
d_4,11047,7381,6984,3051,2324,4232,1440,2258,2954,2169,...,18878,2531,138,2732,1133,1508,3606,13764,25412,NaT
d_5,9925,5912,3309,2630,1942,3817,1536,1694,2492,1726,...,14603,1714,100,1802,927,1209,2869,10525,19146,NaT


In [73]:
states.shape

(1941, 25)

In [63]:
cal

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [40]:
state = sales.state_id.unique()
store = sales.store_id.unique()
dept = sales.dept_id.unique()
cat = sales.cat_id.unique()

In [41]:
total = {'total': list(state)}
state_h = {k: [v for v in store if v.startswith(k)] for k in state}
store_h = {k: [v for v in cat if v.startswith(k)] for k in store}
dept_h = {k: [v for v in dept if v.startswith(k)] for k in cat}

In [42]:
hierarchy = {**total, **state_h, **store_h, **dept_h}
hierarchy

{'total': ['CA', 'TX', 'WI'],
 'CA': ['CA_1', 'CA_2', 'CA_3', 'CA_4'],
 'TX': ['TX_1', 'TX_2', 'TX_3'],
 'WI': ['WI_1', 'WI_2', 'WI_3'],
 'CA_1': [],
 'CA_2': [],
 'CA_3': [],
 'CA_4': [],
 'TX_1': [],
 'TX_2': [],
 'TX_3': [],
 'WI_1': [],
 'WI_2': [],
 'WI_3': [],
 'HOBBIES': ['HOBBIES_1', 'HOBBIES_2'],
 'HOUSEHOLD': ['HOUSEHOLD_1', 'HOUSEHOLD_2'],
 'FOODS': ['FOODS_1', 'FOODS_2', 'FOODS_3']}

In [43]:
ht = HierarchyTree.from_nodes(nodes=hierarchy, df=states)

In [44]:
ht.children

[- CA
    |- CA_1
    |- CA_2
    |- CA_3
    - CA_4,
 - TX
    |- TX_1
    |- TX_2
    - TX_3,
 - WI
    |- WI_1
    |- WI_2
    - WI_3]

In [45]:
ht.children[0].children[2]

- CA_3

In [49]:
#fit to HTSregressor 
clf = HTSRegressor(model='prophet', revision_method='OLS', n_jobs=12)
model = clf.fit(states, hierarchy)

Fitting models:   0%|          | 0/14 [00:05<?, ?it/s]


ParserError: Unknown string format: d_1